# Задание 1. Прямые методы решения линейных систем

In [3]:
from latexifier import latexify
from IPython.display import display, Markdown
import numpy as np
from typing import Callable, List
import warnings
warnings.filterwarnings('ignore')

In [33]:
class Matrix(np.ndarray):
    """extends np.ndarray"""

    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def inverse_rows(self, i: int, j: int):
        """Swaps i-th and j-th rows of a matrix"""
        self[i], self[j] = self[j].copy(), self[i].copy()

    def inverse_cols(self, i: int, j: int):
        """Swaps i-th and j-th columns of a matrix"""
        self = self.transpose()
        self.inverse_rows(i, j)
        self = self.transpose()
    
    def tex(self) -> str:
        """Latexifies the matrix rounding up values up to 6 decimal values"""
        return latexify(self)

def append(M: np.ndarray, N: np.ndarray) -> Matrix:
    """Appends two numpy arrays with axis=1"""
    return Matrix(np.append(arr=np.asarray(M), values=N, axis=1))

def print_tex(*argv) -> None:
    """Displays a LaTex Markdown output"""
    res = ' '.join(['$$'] + [arg for arg in argv] + ['$$'])
    display(Markdown(res))

def get_latex_column(arg: str, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of {arg} with indeces form 1 to {n}"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = arg + "_{" + str(i+1) + "}" + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)

def get_exact_column(col: Matrix, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of {col} elements"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = str(float(col[i][0])) + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)

def print_system_output(A: Matrix, B: Matrix, X: Matrix, n: int) -> None:
    B_ = np.matmul(A, X)
    print_tex('X^* = ', get_latex_column("x^*", n), '=', X.tex())
    print_tex(r'A \times X^* = ', A.tex(), X.tex(), '=', B_.tex(), '= B^*')
    print_tex('B = ', B.tex(), r'\stackrel{?}{=}', B_.tex(), '= B^*')
    print_tex(r'B - B^* = ', get_exact_column(B - B_, n))


In [5]:
s = get_latex_column("x", 5)
print_tex(s, s)

$$ \begin{pmatrix}
x_{1}\\
x_{2}\\
x_{3}\\
x_{4}\\
x_{5}
\end{pmatrix} \begin{pmatrix}
x_{1}\\
x_{2}\\
x_{3}\\
x_{4}\\
x_{5}
\end{pmatrix} $$

In [22]:
n = int(input()) # Размерность матрицы A
A = Matrix(np.random.rand(n, n) * 10)
B = Matrix(np.random.rand(n, 1) * 10)
print_tex('A = ', A.tex(), ',~~', 'B = ', B.tex())
print_tex(A.tex(), get_latex_column('x', n), '=', B.tex())

$$ A =  \begin{pmatrix} 0.309521 & 4.328177 & 6.041022 & 7.488171 \\ 9.835131 & 8.308046 & 2.306726 & 0.219779 \\ 9.387457 & 7.985897 & 5.687012 & 2.130799 \\ 7.847413 & 2.756732 & 7.900442 & 8.086316 \end{pmatrix} ,~~ B =  \begin{pmatrix} 6.453322 \\ 6.154414 \\ 2.363097 \\ 9.942639 \end{pmatrix} $$

$$ \begin{pmatrix} 0.309521 & 4.328177 & 6.041022 & 7.488171 \\ 9.835131 & 8.308046 & 2.306726 & 0.219779 \\ 9.387457 & 7.985897 & 5.687012 & 2.130799 \\ 7.847413 & 2.756732 & 7.900442 & 8.086316 \end{pmatrix} \begin{pmatrix}
x_{1}\\
x_{2}\\
x_{3}\\
x_{4}
\end{pmatrix} = \begin{pmatrix} 6.453322 \\ 6.154414 \\ 2.363097 \\ 9.942639 \end{pmatrix} $$

### Решение методом Гаусса единственного деления

In [34]:
def SingleGauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    A_ = A.copy()
    B_ = B.copy()
    X = [[0] for _ in range(n)]
    for k in range(n):
        for j in range(k+1, n):
            d = A_[j][k] / A_[k][k]
            for i in range(k, n):
                A_[j][i] = A_[j][i] - d * A_[k][i]
            B_[j] -= d * B_[k][0]
    for k in range(n-1, -1, -1):
        d = 0
        for j in range(k, n):
            s = A_[k][j] * X[j][0]
            d += s
        X[k][0] = (B_[k][0] - d) / A_[k][k]
    return Matrix(np.array(X))


print_system_output(A, B, SingleGauss(A, B, n), n)


$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix} 0.920138 \\ 0.270598 \\ -2.484280 \\ 2.671536 \end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 0.309521 & 4.328177 & 6.041022 & 7.488171 \\ 9.835131 & 8.308046 & 2.306726 & 0.219779 \\ 9.387457 & 7.985897 & 5.687012 & 2.130799 \\ 7.847413 & 2.756732 & 7.900442 & 8.086316 \end{pmatrix} \begin{pmatrix} 0.920138 \\ 0.270598 \\ -2.484280 \\ 2.671536 \end{pmatrix} = \begin{pmatrix} 6.453322 \\ 6.154414 \\ 2.363097 \\ 9.942639 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix} 6.453322 \\ 6.154414 \\ 2.363097 \\ 9.942639 \end{pmatrix} \stackrel{?}{=} \begin{pmatrix} 6.453322 \\ 6.154414 \\ 2.363097 \\ 9.942639 \end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
2.6645352591003757e-15\\
2.6645352591003757e-15\\
7.283063041541027e-14\\
-7.815970093361102e-14
\end{pmatrix} $$

### Решение методом Гаусса с выбором главного элемента

In [35]:
def ModifiedGauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    AB = append(A, B)
    X = [[0] for _ in range(n)]
    for k in range(n):
        p = k # главный элемент
        for m in range(k+1, n):
            if abs(AB[p][k]) < abs(AB[m][k]):
                p = m # индекс строки, содержащей максимальный элемент в столбце
        AB.inverse_rows(k, p) # перестановка строк
        for m in range(k+1, n):
            c = AB[m][k] / AB[k][k]
            for i in range(n+1):
                AB[m][i] -= c * AB[k][i]
        X[n-1][0] = AB[n-1][n] / AB[n-1][n-1]
        for k in range(n-1, -1, -1):
            s = 0
            for i in range(k+1, n):
                s += AB[k][i] * X[i][0]
            X[k][0] = (AB[k][n] - s) / AB[k][k]
    return Matrix(np.array(X))

print_system_output(A, B, ModifiedGauss(A, B, n), n)

$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix} 0.920138 \\ 0.270598 \\ -2.484280 \\ 2.671536 \end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 0.309521 & 4.328177 & 6.041022 & 7.488171 \\ 9.835131 & 8.308046 & 2.306726 & 0.219779 \\ 9.387457 & 7.985897 & 5.687012 & 2.130799 \\ 7.847413 & 2.756732 & 7.900442 & 8.086316 \end{pmatrix} \begin{pmatrix} 0.920138 \\ 0.270598 \\ -2.484280 \\ 2.671536 \end{pmatrix} = \begin{pmatrix} 6.453322 \\ 6.154414 \\ 2.363097 \\ 9.942639 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix} 6.453322 \\ 6.154414 \\ 2.363097 \\ 9.942639 \end{pmatrix} \stackrel{?}{=} \begin{pmatrix} 6.453322 \\ 6.154414 \\ 2.363097 \\ 9.942639 \end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
2.6645352591003757e-15\\
0.0\\
1.7763568394002505e-15\\
3.552713678800501e-15
\end{pmatrix} $$